In [1]:
import pyranges as pr
import pandas as pd
import numpy as np
from define_helpers import select_rep_site, _define_cryptic_status, select_rep_prox_site, select_rep_five_end, construct_bed_name
import os


# approach

### Input last exons - selecting representative PAS/last exon:

le_ids can have multiple predictions, need to select a single one.
Strategy:
(starting with novel last exons used for quantification) 
1. Prefer events that match atlas
2. if multiple, select min atlas distance
3. If matches, s





## To-dos:
- fix updating to atlas for -ve strand matching pas
- run selection for all novel 3'UTR extensions
- Implement selection for annotated proximal sites of 3'UTR extensions (i.e. most distal annotated site)


# 

In [2]:
# input novel last exons used to generate combined reference of last exons
novel_le = pr.read_gtf("../data/papa/2023-03-29_papa_i3_cortical_upf1_zanovello_overlap_annotated.gtf")
# dexseq results df (used to extract cryptic events of each class)
dexseq_df = pd.read_csv("../data/papa/2023-05-24_i3_cortical_zanovello.all_datasets.dexseq_apa.results.processed.cleaned.tsv", sep="\t")

cryptics_df = pd.read_csv("../../preprocessing/processed/cryptics_summary_all_events.tsv", sep="\t")

# refeence GTF used to identify novel last exons, quantify vs ref
ref_gtf = pr.read_gtf("../data/reference_filtered.gtf")
# 
tx2le = pd.read_csv("../data/papa/novel_ref_combined.tx2le.tsv", sep="\t")

# last exon quantification regions used as input to Salmon
quant_uniq_le = pr.read_gtf("../data/papa/novel_ref_combined.quant.last_exons.gtf")


info_df = pd.read_csv("../data/papa/novel_ref_combined.info.tsv", sep="\t")


In [3]:
print(quant_uniq_le.columns)
quant_uniq_le.subset(lambda df: ~df.transcript_id.str.startswith("ENST"))

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'gene_name', 'transcript_id', 'exon_number',
       'region_rank', 'Start_ref', 'End_ref', 'transcript_id_ref',
       '3p_extension_length', 'event_type', 'ref_gene_id', 'ref_gene_name',
       'le_number', 'le_id'],
      dtype='object')


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24476667,.,+,.,PAPA.NT_19074713_S59.398,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
1,chr1,.,exon,24468985,24476667,.,+,.,PAPA.ctrl_ctrl_3.424,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
2,chr1,.,exon,24468985,24475256,.,+,.,PAPA.TDP43_19065409_S29.438,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
3,chr1,.,exon,24468985,24473773,.,+,.,PAPA.Cont-B_S2.334,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
4,chr1,.,exon,24468985,24474852,.,+,.,PAPA.DZ_curves_1_3.360,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_extension,ENSG00000001461.17,NIPAL3,6.0,ENSG00000001461.17_6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36889,chrY,.,exon,13234576,13234826,.,-,.,PAPA.NT_19074729_S12.25635,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36890,chrY,.,exon,13234576,13234826,.,-,.,PAPA.TDP43-F_S6.20683,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36891,chrY,.,exon,13234576,13234826,.,-,.,PAPA.TDP43_19065413_S19.26168,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7
36892,chrY,.,exon,13234583,13234826,.,-,.,PAPA.DZ_curves_0_2.21889,NaN,...,NaN,NaN,NaN,NaN,NaN,last_exon_spliced,ENSG00000183878.16,"UTY,UTY,UTY,UTY,UTY,UTY,UTY,UTY",7.0,ENSG00000183878.16_7


In [4]:
dexseq_df.columns

Index(['experiment_name', 'binID', 'groupID', 'featureID', 'exonBaseMean',
       'dispersion', 'stat', 'pvalue', 'padj', 'UsageCoefficient_base',
       'UsageCoefficient_contrast', 'log2fold_contrast_base', 'le_id',
       'gene.qvalue', 'contrast_name', 'gene_id', 'gene_name', 'event_type',
       'annot_status', 'transcript_id', 'chromosome', 'strand', 'start', 'end',
       'mean_PPAU_base', 'mean_PPAU_treatment', 'delta_PPAU_treatment_control',
       'simple_event_type'],
      dtype='object')

In [5]:
novel_le.columns

Index(['Chromosome', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand',
       'Frame', 'gene_id', 'transcript_id', 'event_type',
       '3p_extension_length', 'atlas_filter', 'nearest_atlas_distance', 'Name',
       'motif_filter', 'pas_motifs', 'condition_id', 'le_id', 'gene_name_ref',
       'gene_id_ref', 'mean_PPAU_base', 'mean_PPAU_treatment',
       'delta_PPAU_treatment_control', 'experiment_id', 'Cluster',
       'experiment_count', 'multiple_datasets', 'max_mean_PPAU'],
      dtype='object')

In [6]:
# summary df of cryptic events following manual validation of bleedthrough events
cryptics_df = pd.read_csv("../../preprocessing/processed/cryptics_summary_all_events_bleedthrough_manual_validation.tsv", sep="\t")

# load in IDs to remove entirely from analysis (remove as cryptic, but make sure nto included in overall background)
with open("../../preprocessing/processed/cryptics_manual_validation_fail_le_ids.txt", "r") as infile:
    manual_validation_fail_ids = [line.rstrip("\n") for line in infile]

# get a dict of set of le_ids in each case
event_type_le_ids = cryptics_df.groupby("simple_event_type")["le_id"].agg(set).to_dict()
event_type_gene_names = cryptics_df.groupby("simple_event_type")["gene_name"].agg(set).to_dict()

for event_type, ids in event_type_le_ids.items():
    print(f"Event type - {event_type} - number of IDs - {len(ids)}")

print(f"Number of cryptic IDs failing manual validation - {len(manual_validation_fail_ids)}")

Event type - bleedthrough - number of IDs - 21
Event type - bleedthrough,distal_3utr_extension - number of IDs - 3
Event type - bleedthrough,spliced - number of IDs - 12
Event type - distal_3utr_extension - number of IDs - 104
Event type - spliced - number of IDs - 119
Number of cryptic IDs failing manual validation - 34


In [7]:
# filter out failed IDs from dexseq df (so not included as background)
dexseq_df = dexseq_df[~dexseq_df["le_id"].isin(manual_validation_fail_ids)]
dexseq_df

,experiment_name,binID,groupID,featureID,exonBaseMean,dispersion,stat,pvalue,padj,UsageCoefficient_base,...,annot_status,transcript_id,chromosome,strand,start,end,mean_PPAU_base,mean_PPAU_treatment,delta_PPAU_treatment_control,simple_event_type
0,brown_i3_cortical,ENSG00000021645.20:E001,ENSG00000021645.20,E001,32.627088,0.044220,90.962787,1.463997e-21,2.250563e-19,1.452069,...,novel,PAPA.TDP-4.18197.2,chr14,+,7.829867e+07,7.830620e+07,0.001914,0.033848,0.031934,spliced
1,brown_i3_cortical,ENSG00000021645.20:E002,ENSG00000021645.20,E002,622.521901,0.022117,135.739619,2.274890e-31,5.061630e-29,12.675156,...,annotated,"ENST00000557594.5,ENST00000428277.6,ENST000002...",chr14,+,7.986104e+39,7.986248e+39,0.998086,0.934776,-0.063310,spliced
2,brown_i3_cortical,ENSG00000021645.20:E002,ENSG00000021645.20,E002,622.521901,0.022117,135.739619,2.274890e-31,5.061630e-29,12.675156,...,annotated,"ENST00000557594.5,ENST00000428277.6,ENST000002...",chr14,+,7.986104e+39,7.986248e+39,0.998086,0.934776,-0.063310,spliced
3,brown_i3_cortical,ENSG00000048649.14:E001,ENSG00000048649.14,E001,163.262346,0.037855,88.319669,5.568698e-21,8.407740e-19,4.515264,...,novel,"PAPA.doxconc_DOX_0075_2.11065.7,PAPA.TDP43-G_S...",chr11,-,7.781315e+71,7.781379e+71,0.090158,0.462779,0.372622,spliced
4,brown_i3_cortical,ENSG00000048649.14:E001,ENSG00000048649.14,E001,163.262346,0.037855,88.319669,5.568698e-21,8.407740e-19,4.515264,...,annotated,"PAPA.doxconc_DOX_0075_2.11065.7,PAPA.TDP43-G_S...",chr11,-,7.781315e+71,7.781379e+71,0.090158,0.462779,0.372622,spliced
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209479,zanovello_skndz_curve_1,ENSG00000280789.2:E002,ENSG00000280789.2,E002,99.077638,12.000000,0.006587,9.353145e-01,9.785953e-01,5.982423,...,novel,PAPA.chx_tdp_CTRL_ctrl_4.15153.2,chr16,+,2.981955e+07,2.982252e+07,0.181220,0.333333,0.152113,distal_3utr_extension
209480,zanovello_skndz_curve_1,ENSG00000280832.2:E001,ENSG00000280832.2,E001,75.778902,0.099478,0.075875,7.829679e-01,9.189656e-01,6.014731,...,annotated,ENST00000626810.1,chr11,-,1.263502e+08,1.263518e+08,0.133144,0.147606,0.014462,spliced
209481,zanovello_skndz_curve_1,ENSG00000280832.2:E002,ENSG00000280832.2,E002,402.882205,0.085732,0.087526,7.673463e-01,9.134343e-01,12.145436,...,annotated,ENST00000629441.2,chr11,-,1.263409e+08,1.263423e+08,0.866856,0.852394,-0.014462,spliced
209482,zanovello_skndz_curve_1,ENSG00000282508.2:E001,ENSG00000282508.2,E001,43.093715,0.060733,0.017535,8.946533e-01,9.665471e-01,4.668928,...,annotated,ENST00000633109.1,chr19,-,2.112510e+05,2.117150e+05,0.678888,0.582547,-0.096341,spliced


In [8]:
# get le_ids that are novel 3'UTR ectensions
le_id_d3utr = set(dexseq_df[dexseq_df.simple_event_type == "distal_3utr_extension"].le_id)
len(le_id_d3utr)

3369

In [9]:
cryp_le_ids = event_type_le_ids["distal_3utr_extension"]
cryp_gene_names = event_type_gene_names["distal_3utr_extension"]

print(f"Number of cryptic 3'UTR extensions - {len(cryp_le_ids)}")
print(f"Number of cryptic 3'UTR extension-containing genes - {len(cryp_gene_names)}")

Number of cryptic 3'UTR extensions - 104
Number of cryptic 3'UTR extension-containing genes - 104


In [10]:
# add le_id used in downstream analysis
novel_le = novel_le.apply(lambda df: df.merge(tx2le, on="transcript_id", how="inner", suffixes=[None, "_quant"]))

In [11]:
# Select representative PAS for distal 3'UTR extension events (novel)
novel_le_rep_d3utr, rep_choices_d3utr = select_rep_site(novel_le.subset(lambda df: df.le_id_quant.isin(set(le_id_d3utr))), id_col="le_id_quant")


print(f"Number of novel 3'UTR extenstion intervals before selecting representative LEs - {len(novel_le.subset(lambda df: df.le_id_quant.isin(set(le_id_d3utr))))}")
print(f"Number of novel 3'UTR extension intervals after selecting representative LEs - {len(novel_le_rep_d3utr)}")
print({dec: len(ids) for dec, ids in rep_choices_d3utr.items()})

Number of novel 3'UTR extenstion intervals before selecting representative LEs - 15810
Number of novel 3'UTR extension intervals after selecting representative LEs - 3398
{'atlas_1_pred': 1549, 'atlas_max_datasets': 534, 'atlas_max_datasets_shortest': 344, 'motif_1_min': 889, 'motif_shortest_min': 53}


In [12]:
# double check if any IDs dropped
le_id_d3utr - set(novel_le_rep_d3utr.le_id_quant)

set()

In [13]:
# now need to select representative proximal site for each distal site, since quantification approach lumps together all annotated proximal sites (and compares them to extension)
# since le_ids are annotated sequentially, know that immediately preceding le_number is the partner le_id
d3utr_le_id_split = novel_le_rep_d3utr.le_id_quant.str.split("_", regex=False, expand=True)
print(d3utr_le_id_split)
d3utr_le_id_split[1] = d3utr_le_id_split[1].astype(int).subtract(1)

# reconstruct le_id (corresponding to proximal le_id)
d3utr_prox_le_id = d3utr_le_id_split[0].str.cat(d3utr_le_id_split[1].astype(str), sep="_")
d3utr_prox_le_id

                       0  1
6     ENSG00000001461.17  6
85    ENSG00000023902.14  3
98    ENSG00000031698.13  3
102   ENSG00000033122.21  5
143    ENSG00000053372.5  3
...                  ... ..
1137  ENSG00000129824.16  2
1798   ENSG00000154620.6  2
2091  ENSG00000165246.15  3
60    ENSG00000012817.16  3
3081   ENSG00000230663.1  3

[3398 rows x 2 columns]


6       ENSG00000001461.17_5
85      ENSG00000023902.14_2
98      ENSG00000031698.13_2
102     ENSG00000033122.21_4
143      ENSG00000053372.5_2
                ...         
1137    ENSG00000129824.16_1
1798     ENSG00000154620.6_1
2091    ENSG00000165246.15_2
60      ENSG00000012817.16_2
3081     ENSG00000230663.1_2
Name: 0, Length: 3398, dtype: object

In [14]:
#subset for annotated last exons with 3'end proximal to 3'UTR extensions
quant_uniq_le_d3utr_prox = quant_uniq_le.subset(lambda df: df.le_id.isin(set(d3utr_prox_le_id.values)))
quant_uniq_le_d3utr_prox


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
1,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
2,chr1,.,exon,150158818,150159404,.,+,.,ENSG00000023902.14,PLEKHO1,...,last,150159404,150159497,ENST00000502767.1,568.0,first_exon_extension,ENSG00000023902.14,PLEKHO1,2.0,ENSG00000023902.14_2
3,chr1,.,exon,150159497,150160065,.,+,.,ENSG00000023902.14,PLEKHO1,...,last,150159404,150159497,ENST00000502767.1,568.0,first_exon_extension,ENSG00000023902.14,PLEKHO1,2.0,ENSG00000023902.14_2
4,chr1,.,exon,150158818,150159404,.,+,.,ENSG00000023902.14,PLEKHO1,...,last,150159404,150159497,ENST00000502767.1,133.0,first_exon_extension,ENSG00000023902.14,PLEKHO1,2.0,ENSG00000023902.14_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12702,chrY,.,exon,19703864,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2
12703,chrY,.,exon,19705416,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2
12704,chrY,.,exon,19705419,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2
12705,chrY,.,exon,19705424,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2


In [15]:
# double check if any cryptic genes dropped
# [le_id.split("_")] prevents trying to unpack each character in string
# list(le_id_d3utr)

missing_ids_prox = set("_".join([i, str(int(n) - 1)]) for le_id in le_id_d3utr for i,n in [le_id.split("_")]) - set(quant_uniq_le_d3utr_prox.le_id)
# go back to original ids
missing_ids = set("_".join([i, str(int(n) + 1)]) for le_id in missing_ids_prox for i,n in [le_id.split("_")])
# set(quant_uniq_le_d3utr_prox.le_id)
missing_ids
# set(len(le_id.split("_")) for le_id in le_id_d3utr)

# - set(quant_uniq_le_d3utr_prox.le_id)

{'ENSG00000126214.22_3',
 'ENSG00000160439.16_4',
 'ENSG00000165156.15_3',
 'ENSG00000176222.9_3',
 'ENSG00000196268.12_4',
 'ENSG00000213923.13_4'}

In [16]:
# are these cryptic?
print(cryptics_df[cryptics_df.le_id.isin(missing_ids)])


Empty DataFrame
Columns: [le_id, gene_name, simple_event_type, n_exper_cryptic, experiment_name, annot_status, chromosome, start, end, strand]
Index: []


In [17]:
# select representative proximal site for partner annotated ALEs (most distal of annotated PAS)
sel_le_d3utr_prox = select_rep_prox_site(quant_uniq_le_d3utr_prox, id_col="le_id")
sel_le_d3utr_prox

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,region_rank,Start_ref,End_ref,transcript_id_ref,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id
0,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
1,chr1,.,exon,24468985,24472976,.,+,.,ENSG00000001461.17,NIPAL3,...,last,"24464120,24464120","24468985,24468985","ENST00000003912.7,ENST00000374399.9","NULL,NULL",last_exon_spliced,ENSG00000001461.17,NIPAL3,5.0,ENSG00000001461.17_5
2,chr1,.,exon,150159497,150160065,.,+,.,ENSG00000023902.14,PLEKHO1,...,last,150159404,150159497,ENST00000502767.1,568.0,first_exon_extension,ENSG00000023902.14,PLEKHO1,2.0,ENSG00000023902.14_2
3,chr1,.,exon,109237730,109238182,.,+,.,ENSG00000031698.13,SARS1,...,last,109237373,109237730,ENST00000468588.1,NULL,first_exon_spliced,ENSG00000031698.13,SARS1,2.0,ENSG00000031698.13_2
4,chr1,.,exon,70121779,70121994,.,+,.,ENSG00000033122.21,LRRC7,...,last,70107826,70121779,ENST00000415775.2,NULL,last_exon_spliced,ENSG00000033122.21,LRRC7,4.0,ENSG00000033122.21_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4387,chrY,.,exon,13705224,13706024,.,+,.,ENSG00000154620.6,TMSB4Y,...,last,13704435,13705224,ENST00000284856.4,NULL,first_exon_spliced,ENSG00000154620.6,TMSB4Y,1.0,ENSG00000154620.6_1
4388,chrY,.,exon,14840412,14845650,.,+,.,ENSG00000165246.15,NLGN4Y,...,last,"14830519,14830519,14830519,14830519","14840412,14840412,14840412,14840412","ENST00000339174.9,ENST00000382868.5,ENST000003...","NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000165246.15,NLGN4Y,2.0,ENSG00000165246.15_2
4389,chrY,.,exon,19703864,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2
4390,chrY,.,exon,19703864,19706345,.,-,.,ENSG00000012817.16,KDM5D,...,last,"19706345,19706345,19706345,19706345,19706345","19706440,19706440,19706440,19706440,19706440","ENST00000317961.9,ENST00000382806.6,ENST000004...","NULL,NULL,NULL,NULL,NULL",last_exon_spliced,ENSG00000012817.16,KDM5D,2.0,ENSG00000012817.16_2


In [18]:
# check no further cryptics/le_ids dropped
set("_".join([i, str(int(n) - 1)]) for le_id in le_id_d3utr for i,n in [le_id.split("_")]) - set(sel_le_d3utr_prox.le_id)

{'ENSG00000126214.22_2',
 'ENSG00000160439.16_3',
 'ENSG00000165156.15_2',
 'ENSG00000176222.9_2',
 'ENSG00000196268.12_3',
 'ENSG00000213923.13_3'}

In [19]:
# same 6 IDs. Don't think losing 6 events from background is going to make a dramatic difference...
# remove missing IDs from novel distal DF, so don't have inbalanced numbers of proximal/distal events
novel_le_rep_d3utr = novel_le_rep_d3utr.subset(lambda df: ~df.le_id_quant.isin(missing_ids))

In [20]:


# Alternative definition of background (more conservative)
# not significant in any SH-SY5Y dataset, but expressed/evaluated in at least one of them (so know is multi-ALE in at least one dataset)
# NB: extra-check that expressed/evaluated in all 3? Then confident that is a multi-ALE gene robustly expressed/detected
shsy5y_datasets = ["zanovello_shsy5y_curve_0075", "zanovello_shsy5y_chx_kd_ctl_vs_ctl_ctl", "brown_shsy5y"]
dexseq_df_shsy5y = dexseq_df.loc[dexseq_df.experiment_name.isin(shsy5y_datasets), :]

# first get an experiment count for each event
gene_n_shsy5y_expressed = (dexseq_df_shsy5y
.groupby("groupID")
["experiment_name"]
.nunique()
)

print("Gene counts for the number of SH-SY5Y datasets in which genes are expressed")
print(gene_n_shsy5y_expressed.value_counts())

# get genes expressed in all SH-SY5Y datasets but with no sig change in any of them 
ns_gene_ids_shsy5y = (dexseq_df_shsy5y.loc[dexseq_df_shsy5y.groupID.isin(set(gene_n_shsy5y_expressed[gene_n_shsy5y_expressed == len(shsy5y_datasets)].index)), :]
.assign(reg_status=lambda df: np.where(df["padj"].le(0.05), 1, 0))
               .groupby("groupID")
               ["reg_status"]
               .sum()
               .loc[lambda x: x == 0]
)

# set of
ns_le_ids_shsy5y = set(dexseq_df_shsy5y.loc[dexseq_df_shsy5y["groupID"].isin(ns_gene_ids_shsy5y.index), "le_id"])


print(f"Number of ns ALE-containing genes expressed in all SH-SY5Y datasets - {len(ns_gene_ids_shsy5y.index)}")
print(f"Number of ns ALE isoforms of genes expressed in all SH-SY5Y datasets - {len(set(ns_le_ids_shsy5y))}")

# le_ids in genes with no differential usage in any dataset that are also expressed/assessed in all datasets
ns_le_ids_shsy5y_all = set(dexseq_df_shsy5y[dexseq_df_shsy5y.le_id.isin(ns_le_ids_shsy5y)]
.groupby("le_id")
["experiment_name"]
.nunique()
.loc[lambda x: x == 3]
.index
)

print("Of the genes containing ALE isoforms, how many datasets are the individual isoforms expressed in?")
print(dexseq_df_shsy5y[dexseq_df_shsy5y.le_id.isin(ns_le_ids_shsy5y)]
.groupby("le_id")
["experiment_name"]
.nunique()
.value_counts()
)

print(f"Number of ns ALE isoforms expressed in all SH-SY5Y datasets - {len(ns_le_ids_shsy5y_all)}")
# (dexseq_df.assign(reg_status=lambda df: np.where(df["padj"].le(0.05), 1, 0))
#                .groupby("groupID")
#                ["reg_status"]
#                .sum()
#                
#              )


Gene counts for the number of SH-SY5Y datasets in which genes are expressed
experiment_name
3    2582
1    1068
2     704
Name: count, dtype: int64
Number of ns ALE-containing genes expressed in all SH-SY5Y datasets - 1577
Number of ns ALE isoforms of genes expressed in all SH-SY5Y datasets - 3624
Of the genes containing ALE isoforms, how many datasets are the individual isoforms expressed in?
experiment_name
3    3304
1     185
2     135
Name: count, dtype: int64
Number of ns ALE isoforms expressed in all SH-SY5Y datasets - 3304


In [21]:
# now want to combine the two with minimal info required for making maps
# 3'end coordinates, gene name, le_id, site type, regulation status

# define site_type columns
sel_le_d3utr_prox.site_type = "proximal"
novel_le_rep_d3utr.site_type = "distal"


In [22]:
# le_ids of novel extensions with no change in any experiment
ns_le_ids_d3utr = set(dexseq_df.loc[(dexseq_df["le_id"].isin(ns_le_ids_shsy5y_all.intersection(le_id_d3utr))), "le_id"].drop_duplicates())


# now get le_ids of partner proximal les
ns_le_ids_d3utr_prtnr = set(["_".join([le_id.split("_")[0], str(int(le_id.split("_")[1]) - 1)]) for le_id in ns_le_ids_d3utr])
print(len(ns_le_ids_d3utr) == len(ns_le_ids_d3utr_prtnr))
ns_le_ids_d3utr_comb = ns_le_ids_d3utr.union(ns_le_ids_d3utr_prtnr)
len(ns_le_ids_d3utr_comb)

True


1636

In [23]:
# ensure proximal partner of cryptic les are labelled
cryp_le_ids_prtnr = set(["_".join([le_id.split("_")[0], str(int(le_id.split("_")[1]) - 1)]) for le_id in list(cryp_le_ids)])
cryp_le_ids_comb = cryp_le_ids.union(cryp_le_ids_prtnr)

In [24]:
sel_le_d3utr_prox = sel_le_d3utr_prox.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_le_ids_comb, ns_le_ids_d3utr_comb, id_col="le_id"))
novel_le_rep_d3utr = novel_le_rep_d3utr.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_le_ids_comb, ns_le_ids_d3utr_comb, id_col="le_id_quant"))



# sel_le_d3utr_prox = sel_le_d3utr_prox.assign("cryptic_status", lambda df: pd.Series(np.where(df.gene_id.isin(cryp_gene_ids_d3utr.index), "cryptic", "background"), index=df.index))
# sel_novel_le_d3utr = sel_novel_le_d3utr.assign("cryptic_status", lambda df: pd.Series(np.where(df.gene_id_ref.isin(cryp_gene_ids_d3utr.index), "cryptic", "background"), index=df.index))

In [25]:
print(novel_le_rep_d3utr.cryptic_status.value_counts())
print("\nNumber of duplicated IDs")
# how many IDs are duplicated with distinct regions?
novel_d3utr_dup_ids = set(novel_le_rep_d3utr.drop_duplicate_positions().as_df().le_id_quant.value_counts().loc[lambda x: x > 1].index)
# subset is for le_ids that have more than one entry. Then collapse to a single entry for each le_id and count the number of unique IDs
novel_le_rep_d3utr[novel_le_rep_d3utr.le_id_quant.isin(novel_d3utr_dup_ids)].drop_duplicate_positions().as_df().drop_duplicates(subset="le_id_quant").cryptic_status.value_counts()

cryptic_status
NULL          2486
background     802
cryptic        104
Name: count, dtype: int64

Number of duplicated IDs


cryptic_status
NULL          20
background     9
Name: count, dtype: int64

In [26]:
# number of events in selected proximal df - should be seeing 104 & 802
sel_le_d3utr_prox.cryptic_status.value_counts()

cryptic_status
NULL          3231
background    1012
cryptic        149
Name: count, dtype: int64

In [27]:
# how many events are duplicated when selecting a representative proximal PAS?
sel_d3utr_prox_dup_ids = set(sel_le_d3utr_prox.drop_duplicate_positions().le_id.value_counts().loc[lambda x: x > 1].index)
sel_le_d3utr_prox[sel_le_d3utr_prox.le_id.isin(sel_d3utr_prox_dup_ids)].drop_duplicate_positions().as_df().drop_duplicates(subset="le_id").cryptic_status.value_counts()

cryptic_status
NULL          24
background     6
cryptic        4
Name: count, dtype: int64

In [28]:
print("Number of proximal PAS regions with distinct 5'ends ")
print(sel_le_d3utr_prox.subset(lambda df: df.le_id.isin(sel_d3utr_prox_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.Start.nunique() if (df.Strand == "+").all() else df.End.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )

print("Number of proximal PAS regions with distinct 3'ends")
print(sel_le_d3utr_prox.subset(lambda df: df.le_id.isin(sel_d3utr_prox_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.End.nunique() if (df.Strand == "+").all() else df.Start.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )


Number of proximal PAS regions with distinct 5'ends 
34
Number of proximal PAS regions with distinct 3'ends
0


In [29]:
print("Number of distal PAS regions with distinct 5'ends ")
print(novel_le_rep_d3utr.subset(lambda df: df.le_id_quant.isin(novel_d3utr_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.Start.nunique() if (df.Strand == "+").all() else df.End.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )

print("Number of distal PAS regions with distinct 3'ends")
print(novel_le_rep_d3utr.subset(lambda df: df.le_id_quant.isin(novel_d3utr_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.End.nunique() if (df.Strand == "+").all() else df.Start.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )

Number of distal PAS regions with distinct 5'ends 
0
Number of distal PAS regions with distinct 3'ends
28


Proximal regions - all have different 5'ends/exon starts
Distal regions - all have distinct 3'ends

In [30]:
novel_le_rep_d3utr.subset(lambda df: df.le_id.isin(novel_d3utr_dup_ids)).drop_duplicate_positions()[["le_id", "gene_name_ref", "Name", "cryptic_status"]]

,Chromosome,Start,End,Strand,le_id,gene_name_ref,Name,cryptic_status
0,chr1,70439100,70445530,+,ENSG00000116761.12_2,CTH,1:70445533:+,background
1,chr1,70439100,70445536,+,ENSG00000116761.12_2,CTH,1:70445533:+,background
2,chr1,33494217,33501611,+,ENSG00000121903.15_4,ZSCAN20,1:33501609:+,background
3,chr1,33494217,33501607,+,ENSG00000121903.15_4,ZSCAN20,1:33501609:+,background
4,chr1,44778575,44779122,+,ENSG00000142937.12_2,RPS8,1:44779125:+,NULL
5,chr1,44778575,44779128,+,ENSG00000142937.12_2,RPS8,1:44779125:+,NULL
6,chr1,42775208,42781977,+,ENSG00000164008.17_2,C1orf50,1:42781980:+,NULL
7,chr1,42775208,42781983,+,ENSG00000164008.17_2,C1orf50,1:42781980:+,NULL
8,chr1,173606034,173610215,-,ENSG00000183831.7_3,ANKRD45,1:173606033:-,NULL
9,chr1,173606030,173610215,-,ENSG00000183831.7_3,ANKRD45,1:173606033:-,NULL


In [31]:
# All duplicates are equidistant to the atlas site. For the sake of simplicity and consistency arbitrarily select the most distal site for each le_id
# Distal - select most distal/3' PAS for each le_id
# Proximal - like ALEs, select the most proximal/3' site (i.e. shortest annotated exon 5'end)

sel_le_d3utr_prox = select_rep_five_end(sel_le_d3utr_prox, "le_id")
novel_le_rep_d3utr = select_rep_prox_site(novel_le_rep_d3utr, "le_id_quant")


print("Number of proximal PAS regions with distinct 5'ends ")
print(sel_le_d3utr_prox.subset(lambda df: df.le_id.isin(sel_d3utr_prox_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.Start.nunique() if (df.Strand == "+").all() else df.End.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )


print("Number of distal PAS regions with distinct 3'ends")
print(novel_le_rep_d3utr.subset(lambda df: df.le_id_quant.isin(novel_d3utr_dup_ids))
 .drop_duplicate_positions()
 .as_df()
 .groupby(["le_id", "Strand"])
 .apply(lambda df: df.End.nunique() if (df.Strand == "+").all() else df.Start.nunique()).sort_values()
 .loc[lambda x: x > 1].index.nunique()
 )

Number of proximal PAS regions with distinct 5'ends 
0
Number of distal PAS regions with distinct 3'ends
0


In [32]:
# Construct 'name' string with minimal annotation information
# le_id|gene_name|site_type|cryptic_status

sel_le_d3utr_prox = (sel_le_d3utr_prox
                     .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("ref_gene_name",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         # some ref_gene_name entries have multiple gene names
         lambda df: df["ref_gene_name"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
 .assign("gene_name_common",
                          lambda df: pd.Series(np.where(df["gene_name"].isna(),
                                                        df["ref_gene_name"],
                                                        df["gene_name"])))
 .assign("Name",
                          lambda df: df["le_id"].str.cat(df[["gene_name_common", "site_type", "cryptic_status"]], sep="|"))
                    
 )


novel_le_rep_d3utr = (novel_le_rep_d3utr
                      .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("gene_name_ref",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         lambda df: df["gene_name_ref"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
 .assign("Name",
         lambda df: df["le_id_quant"].str.cat(df[["gene_name_ref", "site_type", "cryptic_status"]], sep="|"))
)


In [33]:
# quick double check that all cryptic events are retained
print(cryp_le_ids_prtnr - set(sel_le_d3utr_prox.subset(lambda df: df.cryptic_status.eq("cryptic")).le_id))
cryp_le_ids - set(novel_le_rep_d3utr.subset(lambda df: df.cryptic_status.eq("cryptic")).le_id_quant)

set()


set()

In [34]:
# generate output bed file of proximal and distal PAS
utr3_out_bed = pr.concat([sel_le_d3utr_prox[["Score", "Name"]],
                          novel_le_rep_d3utr[["Score", "Name"]]]
                          )

print(len(utr3_out_bed))
# drop duplicate entries
utr3_out_bed = utr3_out_bed.drop_duplicate_positions().sort()
utr3_out_bed

2040


,Chromosome,Start,End,Score,Strand,Name
0,chr1,6219235,6220805,.,+,ENSG00000158286.13_3|RNF207|proximal|background
1,chr1,6219235,6221220,.,+,ENSG00000158286.13_4|RNF207|distal|background
2,chr1,11294872,11296049,.,+,ENSG00000120942.14_2|UBIAD1|proximal|background
3,chr1,11294872,11299544,.,+,ENSG00000120942.14_3|UBIAD1|distal|background
4,chr1,15939295,15940456,.,+,ENSG00000065526.12_3|SPEN|proximal|cryptic
...,...,...,...,...,...,...
1789,chrX,149923199,149930081,.,-,ENSG00000197021.9_4|EOLA2|distal|background
1790,chrX,154505228,154506906,.,-,ENSG00000071889.17_3|FAM3A|distal|background
1791,chrX,154506158,154506906,.,-,ENSG00000071889.17_2|FAM3A|proximal|background
1792,chrX,155478419,155491666,.,-,ENSG00000185973.12_5|TMLHE|distal|background


In [35]:
# get event counts for cryptic/bg for each region set
# event_counts = {}
# for nm, gr in {"spliced_all_background": spliced_out_bed,
#                "spliced_shsy5y_background": spliced_out_bed_shsy5y_all,
#                "bleedthrough_all_background": bleed_out_bed,
#                "bleed_out_bed_shsy5y_all": bleed_out_bed_shsy5y_all}.items():
    
#     event_counts[nm] = gr.Name.str.split("|", expand=True)[3].rename("reg_status").value_counts()

event_counts = {region: utr3_out_bed.subset(lambda df: df.Name.str.contains(region, regex=False)).Name.str.split("|", expand=True)[3].rename("reg_status").value_counts() 
                for region in ["proximal", "distal"]}



event_counts_df = pd.concat(event_counts, names=["event_type"]).reset_index()
event_counts_df

,event_type,reg_status,count
0,proximal,background,793
1,proximal,cryptic,104
2,distal,background,793
3,distal,cryptic,104


In [36]:
if not os.path.exists("../processed/iclip_regions"):
    os.makedirs("../processed/iclip_regions")
    
utr3_out_bed.to_bed("../processed/iclip_regions/2023-12-03_papa_cryptic_d3utr.background_shsy5y.last_exons.bed")
# proximal 3'UTRs
utr3_out_bed.subset(lambda df: df.Name.str.contains("proximal", regex=False)).to_bed("../processed/iclip_regions/2023-12-03_papa_cryptic_d3utr.background_shsy5y.last_exons.proximal.bed")
# distal 3'UTRs
utr3_out_bed.subset(lambda df: df.Name.str.contains("distal", regex=False)).to_bed("../processed/iclip_regions/2023-12-03_papa_cryptic_d3utr.background_shsy5y.last_exons.distal.bed")

# also make one with just polYA sites
utr3_out_bed.three_end().to_bed("../processed/iclip_regions/2023-12-03_papa_cryptic_d3utr.background_shsy5y.pas.bed")
# just proximal
utr3_out_bed.three_end().subset(lambda df: df.Name.str.contains("proximal", regex=False)).to_bed("../processed/iclip_regions/2023-12-03_papa_cryptic_d3utr.background_shsy5y.pas.proximal.bed")
# just distal
utr3_out_bed.three_end().subset(lambda df: df.Name.str.contains("distal", regex=False)).to_bed("../processed/iclip_regions/2023-12-03_papa_cryptic_d3utr.background_shsy5y.pas.distal.bed")

event_counts_df.to_csv("../processed/iclip_regions/2023-12-03_papa_d3utr.event_counts.tsv", sep="\t", header=True,index=False)

In [37]:
# also make a BED with distal bins
utr3_out_bed_prox = utr3_out_bed.subset(lambda df: df.Name.str.contains("proximal", regex=False))
utr3_out_bed_dist = utr3_out_bed.subset(lambda df: df.Name.str.contains("distal", regex=False))

print(utr3_out_bed_dist)
# uniq region of distal 3'UTR
utr3_out_bed_dist_bin = utr3_out_bed_dist.subtract(utr3_out_bed_prox, strandedness="same")
utr3_out_bed_dist_bin

+--------------+-----------+-----------+------------+--------------+-------+
| Chromosome   | Start     | End       | Score      | Strand       | +1    |
| (category)   | (int64)   | (int64)   | (object)   | (category)   | ...   |
|--------------+-----------+-----------+------------+--------------+-------|
| chr1         | 6219235   | 6221220   | .          | +            | ...   |
| chr1         | 11294872  | 11299544  | .          | +            | ...   |
| chr1         | 15939295  | 15940477  | .          | +            | ...   |
| chr1         | 19258680  | 19264037  | .          | +            | ...   |
| ...          | ...       | ...       | ...        | ...          | ...   |
| chrX         | 123600573 | 123601338 | .          | -            | ...   |
| chrX         | 149923199 | 149930081 | .          | -            | ...   |
| chrX         | 154505228 | 154506906 | .          | -            | ...   |
| chrX         | 155478419 | 155491666 | .          | -            | ...   |

,Chromosome,Start,End,Score,Strand,Name
0,chr1,6220805,6221220,.,+,ENSG00000158286.13_4|RNF207|distal|background
1,chr1,11296049,11299544,.,+,ENSG00000120942.14_3|UBIAD1|distal|background
2,chr1,15940456,15940477,.,+,ENSG00000065526.12_4|SPEN|distal|cryptic
3,chr1,19260128,19264037,.,+,ENSG00000053372.5_3|MRTO4|distal|background
4,chr1,24472976,24476667,.,+,ENSG00000001461.17_6|NIPAL3|distal|background
...,...,...,...,...,...,...
935,chrX,110673482,110673855,.,-,ENSG00000101938.15_2|CHRDL1|distal|background
936,chrX,117894639,117897812,.,-,ENSG00000003096.14_2|KLHL13|distal|background
937,chrX,119584579,119588336,.,-,ENSG00000186416.18_2|NKRF|distal|background
938,chrX,154505228,154506158,.,-,ENSG00000071889.17_3|FAM3A|distal|background


In [38]:
utr3_out_bed_dist_bin.to_bed("../processed/iclip_regions/2023-12-03_papa_cryptic_d3utr.background_shsy5y.last_exon_bins.distal.bed")